<a href="https://colab.research.google.com/drive/1wILICRGQE5w2tTMPUZNArp-i19UcUhJW#scrollTo=mWQoXid6FpqA/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Photo Editor using Hand Gestures




In [ ]:
import numpy as np
import cv2 as cv
from tkinter import *
from tkinter import colorchooser
import datetime
import math

In [ ]:
background = None
moveArray = []
a = False
b = False

In [ ]:
def getmask(img):
    hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80], dtype="uint8")
    upper = np.array([20, 255, 255], dtype="uint8")
    skinRegionHSV = cv.inRange(hsvim, lower, upper)
    blurred = cv.blur(skinRegionHSV, (2, 2))
    ret, thresh = cv.threshold(blurred, 0, 255, cv.THRESH_BINARY)
    # cv.imshow("thresh", thresh)
    global background
    if background is None:
        background = img
    return thresh

In [ ]:
def getcnthull(mask_img):
    contours, hierarchy = cv.findContours(mask_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = max(contours, key=lambda x: cv.contourArea(x))
    hull = cv.convexHull(contours)
    return contours, hull

In [ ]:
def getdefects(contours):
    hull = cv.convexHull(contours, returnPoints=False)
    defects = cv.convexityDefects(contours, hull)
    return defects

In [ ]:
def gettypemove(hull):
    global moveArray
    global a
    global b
    if cv.waitKey(1) & 0xFF == ord('a'):
        a = True
    if cv.waitKey(1) & 0xFF == ord('b'):
        b = True

    top = tuple(hull[hull[:, :, 1].argmin()][0])
    bottom = tuple(hull[hull[:, :, 1].argmax()][0])
    left = tuple(hull[hull[:, :, 0].argmin()][0])
    right = tuple(hull[hull[:, :, 0].argmax()][0])
    x1 = (top[0]+bottom[0])/2
    x2 = (left[0]+right[0])/2
    x = (x1+x2)/2
    y1 = (top[1]+bottom[1])/2
    y2 = (right[1]+left[1])/2
    y = (y1+y2)/2
    typeMove = 0
    if (a is True) and (b is False):
        moveArray.append((x,y))

    elif (a is True) and (b is True):
        a = False
        b = False
        print(moveArray)
        c = int(moveArray.__len__()/4)
        esx = moveArray[0][0]-moveArray[-1][0]
        esy = moveArray[0][1]-moveArray[-1][1]
        cx = moveArray[c][0]-moveArray[-c][0]
        cy = moveArray[c][1]-moveArray[-c][1]
        print(esx)
        print(esy)
        esxBool = (esx < 100) and (esx > -100)
        esyBool = (esy < 100) and (esy > -100)
        if esxBool:
            if esyBool:
                typeMove = 1
                print("Move in Circle")
        if not esxBool:
            if esyBool:
                typeMove = 2
                print("Move in Line 1")
        if esxBool:
            if not esyBool:
                typeMove = 3
                print("Move in Line 2")
        if not esxBool:
            if not esyBool:
                if moveArray[0][1] > moveArray[-1][1] :
                    typeMove = 4
                elif moveArray[0][1] < moveArray[-1][1] :
                    typeMove = 5
    return typeMove

In [ ]:
def getTransform(image,typeMove):
    result = image
    if typeMove == 1 :
        print(typeMove)
        rows,cols,c = image.shape
        M=cv.getRotationMatrix2D((cols/2,rows/2), 90,1)
        result = cv.warpAffine(image,M,(cols,rows))
    elif typeMove == 2 :
        print(typeMove)
        result = cv.flip(image,1)
    elif typeMove == 3 :
        print(typeMove)
        result = cv.flip(image,0)
    elif typeMove == 4 :
        print(typeMove)
        img = cv.resize(image, None, fx=2, fy=2)
        result = img[360:1080,360:1080]
    elif typeMove == 5 :
        print(typeMove)
        img = cv.resize(image, None, fx=0.5, fy=0.5)
        for i in range(result.shape[0]):
            for j in range(result.shape[1]):
                result[i,j] = [0,0,0]
        for i in range(180,540):
            for j in range(180,540):
                result[i,j]=img[i-180,j-180]
    return result

In [ ]:
traslateBool = False

def getTransalte(image,hull):
    global traslateBool
    if cv.waitKey(1) & 0xFF == ord('t'):
        traslateBool = not traslateBool
    result = image
    if traslateBool is True:
        top = tuple(hull[hull[:, :, 1].argmin()][0])
        bottom = tuple(hull[hull[:, :, 1].argmax()][0])
        left = tuple(hull[hull[:, :, 0].argmin()][0])
        right = tuple(hull[hull[:, :, 0].argmax()][0])
        x1 = (top[0]+bottom[0])/2
        x2 = (left[0]+right[0])/2
        x = (x1+x2)/2
        y1 = (top[1]+bottom[1])/2
        y2 = (right[1]+left[1])/2
        y = (y1+y2)/2
        dxc = x - 360
        dyc = y - 360
        M = np.float32([[1, 0, dxc],
                [0, 1, dyc]])
        result = cv.warpAffine(image,M,(720,720))
    return result

In [ ]:
color_code = [0,0,0]

def choose_color():
    global color_code
    color = colorchooser.askcolor(title ="Choose color")
    color_code = list(reversed(list(color[0])))

def getColor():
    root = Tk()
    button = Button(root, text = "Select Color",
                    command = choose_color)
    button.pack()
    root.geometry("30x30")
    root.mainloop()

Ds = 3

def drawImage(image,hull):
    global Ds
    if cv.waitKey(1) & 0xFF == ord('c'):
        getColor()
    if (cv.waitKey(1) & 0xFF == ord('m')) and (Ds <= 97):
        Ds += 3
    if (cv.waitKey(1) & 0xFF == ord('n')) and (Ds >= 3):
        Ds -= 3
    y , x = tuple(hull[hull[:, :, 1].argmin()][0])
    w , h , c = image.shape
    s = Ds
    if ((x > 0) and (x < w)) and ((y > 0) and (y < h)):
        image[x:x+s, y:y+s] = color_code
    return image

In [ ]:
Rs = 3

def drawDelete(lastImage,image,hull):
    global Rs
    if (cv.waitKey(1) & 0xFF == ord('m')) and (Rs <= 97):
        Rs += 3
    if (cv.waitKey(1) & 0xFF == ord('n')) and (Rs >= 3):
        Rs -= 3
    y , x = tuple(hull[hull[:, :, 1].argmin()][0])
    w , h , c = image.shape
    s = Rs
    if ((x > 0) and (x < w)) and ((y > 0) and (y < h)):
        image[x:x+s,y:y+s] = lastImage[x:x+s,y:y+s]
    return image

In [ ]:
warpBool = False

def getWarp(image,hull):
    global warpBool
    if cv.waitKey(1) & 0xFF == ord('w'):
        warpBool = not warpBool
    result = image
    if warpBool is True:
        top = tuple(hull[hull[:, :, 1].argmin()][0])
        bottom = tuple(hull[hull[:, :, 1].argmax()][0])
        left = tuple(hull[hull[:, :, 0].argmin()][0])
        right = tuple(hull[hull[:, :, 0].argmax()][0])
        x1 = (top[0]+bottom[0])/2
        x2 = (left[0]+right[0])/2
        x = (x1+x2)/2
        y1 = (top[1]+bottom[1])/2
        y2 = (right[1]+left[1])/2
        y = (y1+y2)/2
        dxc = (x - 360)/5
        dyc = (y - 360)/5
        result = np.zeros(image.shape, dtype=image.dtype)
        rows, cols, c = image.shape
        for i in range(rows):
            for j in range(cols):
                offset_x = int(5.0 * math.sin(2 * 3.14 * i / dxc))
                offset_y = int(5.0 * math.cos(2 * 3.14 * j / dyc))
                if i+offset_y < rows and j+offset_x < cols:
                    result[i,j] = image[(i+offset_y)%rows,(j+offset_x)%cols]
                else:
                    result[i,j] = 0
    return result

In [ ]:
startDraw  = False
startDelete = False

def runApp():
    image = cv.imread('F:\My Own\Photo/20220213_091956.jpg')
    # image = cv.imread('Images/imageResult.jpg')
    src = cv.imread('F:\My Own\Photo/20220213_091956.jpg')
    lastImage = cv.imread('F:\My Own\Photo/20220213_091956.jpg')
    cap = cv.VideoCapture(0)
    while cap.isOpened():
        _, img = cap.read()
        img = cv.flip(img,1)
        mask_img = getmask(img)
        contours, hull = getcnthull(mask_img)
        cv.drawContours(img, [contours], -1, (255, 255, 0), 2)
        cv.drawContours(img, [hull], -1, (0, 255, 255), 2)
        defects = getdefects(contours)
        trans = gettypemove(hull)
        image = getTransform(image,trans)
        image = getTransalte(image,hull)
        image = getWarp(image,hull)
        if cv.waitKey(1) & 0xFF == ord('d'):
            global startDraw
            if startDraw is False:
                lastImage = image
                print(startDraw)
            startDraw = not startDraw
        if cv.waitKey(1) & 0xFF == ord('x'):
            global startDelete
            startDelete = not startDelete
        if startDraw is True :
            image = drawImage(image,hull)
        if startDelete is True :
            iamge = drawDelete(lastImage,image,hull)
        if cv.waitKey(1) & 0xFF == ord('s'):
            cv.imwrite("Images/imageResult.jpg",image)
        if cv.waitKey(1) & 0xFF == ord('r'):
            image = src
        # cv.putText(img, "", (0, 50), cv.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0) , 2, cv.LINE_AA)
        if defects is not None:
            cnt = 0
            for i in range(defects.shape[0]):  # calculate the angle
                s, e, f, d = defects[i][0]
                start = tuple(contours[s][0])
                end = tuple(contours[e][0])
                far = tuple(contours[f][0])
                a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                if angle <= np.pi / 2:
                    cnt += 1
                    cv.circle(img, far, 4, [0, 0, 255], -1)
            if cnt > 0:
                cnt = cnt + 1
            cv.putText(img, str(cnt), (0, 50), cv.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0) , 2, cv.LINE_AA)
            # print(str(cnt))
        imWidth = 720
        imHeight = 720
        img = cv.resize(img, (imWidth, imHeight))
        cv.imshow("Image :", img)
        cv.imshow("Edit Image : ", image)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

In [ ]:
runApp()

In [ ]:
esx = -436.5
esy = 246.75
esxBool = (esx < 100) and (esx > -100)
esyBool = (esy < 100) and (esy > -100)
if not esxBool:
    if not esyBool:
        typeMove = 4
        if moveArray[0][1] > moveArray[-1][1] :
            typeMove = 4
        elif moveArray[0][1] < moveArray[-1][1] :
            typeMove = 5